# **Trabajo Final - Laboratorio II**

*Integrantes:* 
* Celis Mendoza, Elemir
* Dellatore, Andrés
* Forconi, Gerónimo 
* Lignini, Maira
* Palomeque, Claudia
* Postiglione, Sebastián

# 1 - Objetivo

Se busca predecir la velocidad a la que se adopta una mascota.
En este caso, la velocidad de adopción se determina por la rapidez con la que todas las mascotas del grupo son adoptadas.

Los datos que se disponen incluyen información de texto, tabulares y datos de imágenes.

# 2 - Información disponible

## Variables dataset inicial

* PetID - Identificador único en forma de hash del perfil de la mascota
* Tipo - Tipo de animal (1 = Perro, 2 = Gato)
* Nombre - Nombre de la mascota (Vacío si no tiene nombre)
* Edad - Edad de la mascota cuando se registró, en meses
* Raza1 - Raza principal de la mascota 
* Raza2 - Raza secundaria de la mascota, si es de raza mixta 
* Género - Género de la mascota (1 = Macho, 2 = Hembra, 3 = Mixto, si el perfil representa a un grupo de mascotas)
* Color1 - Color 1 de la mascota 
* Color2 - Color 2 de la mascota 
* Color3 - Color 3 de la mascota 
* TamañoMaduro - Tamaño en la madurez (1 = Pequeño, 2 = Mediano, 3 = Grande, 4 = Extra Grande, 0 = No Especificado)
* LongitudPelaje - Longitud del pelaje (1 = Corto, 2 = Mediano, 3 = Largo, 0 = No Especificado)
* Vacunado - Si la mascota ha sido vacunada (1 = Sí, 2 = No, 3 = No Estoy Seguro)
* Desparasitado - Si la mascota ha sido desparasitada (1 = Sí, 2 = No, 3 = No Estoy Seguro)
* Esterilizado - Si la mascota ha sido esterilizada / castrada (1 = Sí, 2 = No, 3 = No Estoy Seguro)
* Salud - Condición de salud (1 = Saludable, 2 = Lesión Menor, 3 = Lesión Grave, 0 = No Especificado)
* Cantidad - Número de mascotas representadas en el perfil
* Tarifa - Tarifa de adopción (0 = Gratis)
* Estado - Ubicación en el estado de Malasia 
* IDRescatador - Identificador único en forma de hash del rescatador
* CantidadVideos - Total de videos subidos para esta mascota
* CantidadFotos - Total de fotos subidas para esta mascota
* Descripción - Descripción escrita del perfil de esta mascota. El idioma principal utilizado es el inglés, con algunas descripciones en malayo o chino.

## Variable Target

*AdoptionSpeed* - Velocidad de adopción (categorizada).

0 - La mascota fue adoptada el mismo día en que fue listada.

1 - La mascota fue adoptada entre 1 y 7 días (1ra semana) después de ser listada.

2 - La mascota fue adoptada entre 8 y 30 días (1er mes) después de ser listada.

3 - La mascota fue adoptada entre 31 y 90 días (2do y 3er mes) después de ser listada.

4 - No hubo adopción después de 100 días de ser listada. (En este conjunto de datos, no hay mascotas que hayan esperado entre 90 y 100 días). 

## Información adicional

* Metadatos de las imágenes
* Datos de Sentimiento


# 3 - Análisis

## Etapa 1 : EDA 

Analisis descriptivo utilizando sólo la información disponible en Train (data/train/train.csv)


In [1]:
# 1. Importo librerias
import os
import numpy as np 
import pandas as pd 
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt

#import cv2 #Para cargar/mostrar/procesar imagenes
import json
import glob #Para listar los archivos en una carpeta

# Librerias propias (contienen los modelos)
import model_metadata
import model_sentiment

# cargar el resto de las librerias necesarias acá





c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False

In [2]:
#Código Auxiliar!
#Para borrar la cache de las librerias propias
#import importlib
#importlib.reload(model_sentiment)
#importlib.reload(model_metadata)

In [3]:
#2. Lectura Dataset inicial 
train_data = pd.read_csv('../data/train/train.csv', index_col='PetID') 

print (train_data.head(10))
# Info del dataset

train_data.info ()
#14.993 registros
#23 variables

# Se presentan variables como númericas cuando en realidad el número en cuestión es una categoría.
# Hay que transformarla 



           Type                     Name  Age  Breed1  Breed2  Gender  Color1  \
PetID                                                                           
86e1089a3     2                   Nibble    3     299       0       1       1   
6296e909a     2              No Name Yet    1     265       0       1       1   
3422e4906     1                   Brisco    1     307       0       1       2   
5842f1ff5     1                     Miko    4     307       0       2       1   
850a43f90     1                   Hunter    1     307       0       1       1   
d24c30b4b     2                      NaN    3     266       0       2       5   
1caa6fcdb     2                    BULAT   12     264     264       1       1   
97aa9eeac     1  Siu Pak & Her 6 Puppies    0     307       0       2       1   
c06d167ca     2                      NaN    2     265       0       2       6   
7a0942d61     2                    Kitty   12     265       0       2       1   

           Color2  Color3  

In [4]:
#3. Transformación de los datos

train_data['Type'] = train_data['Type'].astype('category')
train_data['Breed1'] = train_data['Breed1'].astype('category')
train_data['Breed2'] = train_data['Breed2'].astype('category')
train_data['Gender'] = train_data['Gender'].astype('category')
train_data['Color1'] = train_data['Color1'].astype('category')
train_data['Color2'] = train_data['Color2'].astype('category')
train_data['Color3'] = train_data['Color3'].astype('category')
train_data['MaturitySize'] = train_data['MaturitySize'].astype('category')
train_data['FurLength'] = train_data['FurLength'].astype('category')
train_data['Vaccinated'] = train_data['Vaccinated'].astype('category')
train_data['Dewormed'] = train_data['Dewormed'].astype('category')
train_data['Sterilized'] = train_data['Sterilized'].astype('category')
train_data['Health'] = train_data['Health'].astype('category')
train_data['State'] = train_data['State'].astype('category')
train_data['AdoptionSpeed'] = train_data['AdoptionSpeed'].astype('category')
#train_data.info()--> ok



In [5]:
#4. 
# a. Estadísticas descriptivas (continuas) y tablas de frecuencias (categóricas)

# Separar las columnas continuas y categóricas
columnas_continuas = train_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
columnas_categoricas = train_data.select_dtypes(include=['category']).columns.tolist()

# Estadísticas descriptivas para variables continuas
descripcion_continuas = train_data[columnas_continuas].describe()

# Frecuencias por categorías para variables categóricas
frecuencias_categoricas = {}
for columna in columnas_categoricas:
    frecuencias_categoricas[columna] = train_data[columna].value_counts()

# Imprimir las estadísticas descriptivas para variables continuas
print("Estadísticas descriptivas para variables continuas:")
print(descripcion_continuas)
print()

# Imprimir las frecuencias por categorías para variables categóricas
for columna, frecuencias in frecuencias_categoricas.items():
    print(f"Frecuencias para {columna}:")
    print(frecuencias)
    print()


Estadísticas descriptivas para variables continuas:
                Age      Quantity           Fee      VideoAmt      PhotoAmt
count  14993.000000  14993.000000  14993.000000  14993.000000  14993.000000
mean      10.452078      1.576069     21.259988      0.056760      3.889215
std       18.155790      1.472477     78.414548      0.346185      3.487810
min        0.000000      1.000000      0.000000      0.000000      0.000000
25%        2.000000      1.000000      0.000000      0.000000      2.000000
50%        3.000000      1.000000      0.000000      0.000000      3.000000
75%       12.000000      1.000000      0.000000      0.000000      5.000000
max      255.000000     20.000000   3000.000000      8.000000     30.000000

Frecuencias para Type:
Type
1    8132
2    6861
Name: count, dtype: int64

Frecuencias para Breed1:
Breed1
307    5927
266    3634
265    1258
299     342
264     296
       ... 
99        1
93        1
85        1
81        1
192       1
Name: count, Length: 176

In [6]:

# b. Divido el dataset para poder trabajar con la librería profiling
vbles_continuas = train_data.select_dtypes(include=['float64', 'int64'])
vbles_categoricas = train_data.select_dtypes(include=['category']) 

# Armo sub-dataset de vbles categóricas para analizar
vble_target = vbles_categoricas.filter(['AdoptionSpeed'], axis=1)
vbles_cat_gral = vbles_categoricas.filter(['Type', 'State'], axis=1)
vbles_cat_salud = vbles_categoricas.filter(['Vaccinated', 'Health', 'Sterilized', 'Dewormed' ], axis=1)
vbles_cat_fisicas1 = vbles_categoricas.filter(['Breed1', 'Breed2', 'Color1', 'Color2', 'Color3'], axis=1)
vbles_cat_fisicas2 = vbles_categoricas.filter(['Gender', 'MaturitySize', 'FurLength'], axis=1)


#print(vbles_continuas)
#print(vbles_cat_gral)
#print(vbles_cat_salud)
#print(vbles_cat_fisicas1)
#print(vbles_cat_fisicas2)

In [7]:
# 3. EDA

directorio_actual = os.getcwd()

from ydata_profiling import ProfileReport

EDA_cont = ProfileReport(vbles_continuas)
EDA_cont.to_file(os.path.join(directorio_actual, "EDA/EDA_continuas.html"))

EDA_target = ProfileReport(vble_target)
EDA_target.to_file(os.path.join(directorio_actual, "EDA/EDA_target.html"))

EDA_cat_gral = ProfileReport(vbles_cat_gral)
EDA_cat_gral.to_file(os.path.join(directorio_actual, "EDA/EDA_cat_gral.html"))

EDA_cat_salud = ProfileReport(vbles_cat_salud)
EDA_cat_salud.to_file(os.path.join(directorio_actual, "EDA/EDA_cat_salud.html"))

EDA_cat_fisicas1 = ProfileReport(vbles_cat_fisicas1)
EDA_cat_fisicas1.to_file(os.path.join(directorio_actual, "EDA/EDA_cat_fisicas1.html"))

EDA_cat_fisicas2 = ProfileReport(vbles_cat_fisicas2)
EDA_cat_fisicas2.to_file(os.path.join(directorio_actual, "EDA/EDA_cat_fisicas2.html"))

#del EDA_cont
#del EDA_cat_gral
#del EDA_cat_salud 
#del EDA_cat_fisicas1
#del EDA_cat_fisicas2

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 500.81it/s]


### Análisis exploratorio de los datos

#### Análisis univariado

El dataset no presenta valores faltantes.
En relación al valor “cero”, son valores posibles para las variables indicando la ausencia del atributo que se está midiendo.

*Variables continuas*

Edad

Se presenta una amplia variabilidad en las edades registradas. El rango de edades va desde un mínimo de 0 meses hasta un máximo de 255 meses, lo que indica que se incluyen mascotas de todas las edades. 
La mediana de la edad es de 3 meses, lo que sugiere que la mitad de las mascotas en el conjunto de datos son menores de 3 meses, mientras que la otra mitad son mayores de 3 meses. 
El rango intercuartil (RIQ) de 10 meses indíca que el 50% intermedio de las edades se encuentra en un rango de 2 meses a 12 meses. La desviación estándar de 18.16 indica una dispersión significativa en las edades. 
Se presentan edades extremas en este dataset.

Cantidad (número de perfiles de las mascotas) 

Se evidencia que la mayoría de los valores se concentran en torno a un número bajo, ya que tanto la mediana como los cuartiles (Q1 y Q3) tienen un valor de 1. Además, el rango intercuartil (RIQ) es igual a 0, lo que indica que el 50% intermedio de los datos está en el mismo valor (1).
Sin embargo, la variable muestra tambien una amplia dispersión en los datos. Esto sugiere que, aunque la mayoría de los perfiles de mascotas tienen una cantidad baja, existen algunos valores atípicos o extremos que contribuyen a esta dispersión. El coeficiente de variación (CV) es menor que 1, lo que indica una variabilidad relativamente baja en relación con la media.

Tarifa de adopción

Es evidente que una gran proporción de las adopciones se ofrecen de forma gratuita, ya que el valor mínimo, los percentiles 5, 25, 50 y 75, todos tienen un valor de 0. Esto indica que muchas mascotas se ofrecen sin costo alguno para los adoptantes.
Sin embargo, también se observa una amplia variación en los datos, ya que el rango es de 3000, lo que significa que la tarifa máxima de adopción es 3000 veces mayor que la tarifa mínima. Esto se refleja en la alta desviación estándar y el coeficiente de variación (CV) mayor a 1, lo que indica una dispersión significativa en los datos en relación con la media.
Por lo tanto, aunque la mayoría de las adopciones son gratuitas, existen casos excepcionales donde se cobra una tarifa sustancial por la adopción.

Cantidad de videos

La variable muestra que la gran mayoría de los perfiles de mascotas no incluyen videos, pero también señala una pequeña cantidad de casos excepcionales con un pequeño número de videos. La distribución de datos es altamente sesgada hacia la falta de videos.

Cantidad de fotos

La variable tiene un rango que va desde un mínimo de 0 (indicando que no hay fotos) hasta un máximo de 30 (indicando un valor extremo de fotos). Esto refleja la variedad en la cantidad de fotos que se incluyen en los perfiles de mascotas.
Los percentiles y cuartiles muestran que la mayoría de los perfiles de mascotas tienen al menos algunas fotos, ya que el percentil 5, el primer cuartil (Q1) y la mediana son mayores que 0. El tercer cuartil (Q3) indica que el 75% de los perfiles tienen 5 fotos o menos, y el percentil 95 indica que la gran mayoría de los perfiles tienen 10 fotos o menos.
La distribución de datos tiende a concentrarse en valores bajos, pero permite un número variable de fotos en algunos perfiles.

*Variables categóricas*

En líneas generales podemos decir que:

El Train_data contiene un 54.2% de perros y un 45.8% de gatos. El mayor porcentaje de dichas mascotas corresponden a el estado de Selampor (58.1%) y Kualu Lumpur (25.6%).
En relación a caracteristicas físicas, observamos que en relación a la raza principal predomina la "raza mixta" (perro) (39,5%) y "Siberian Husky" (perro) (24.2%)". En menor medida, se encuentra la "doméstica de pelo medio" (gato) (8.4%). El resto de las razas tienen frecuencias menores al 2.5%.
El color principal de relevancia de las mascotas es el negro (49.5%) y en segundo lugar marrón (25%). El resto de los colores tienen frecuencias menores al 6.5%.
La mayoría de las mascotas son hembras (48.5%). El tamaño predominante es el mediano (68.7%) y la longitud del pelaje es sobresaliente para los pelo corto (58.7%).
En cuestiones de salud, un 48,2% no poseen sus vacunas y un 39.3% si poseen. Del resto de las mascotas, este dato se desconoce. Así mismo, casi la totalidad de las mascotas (96.6%) son saludables. El 67.2% no ha sido castrada y el 56.2% está deparasitada.

Velocidad de adopción

Algunas mascotas (2,7%) son adoptadas inmediatamente. Sin embargo, esto es raro (suerte).
Muchas mascotas no son adoptadas en absoluto (28%) y un 26,9% en el primer mes que se encuentran en el listado.
Las frecuencias en la rápidez de adopción no muestran diferencias significativas entre el resto de los períodos, con lo cuál sería necesario un análisis bivariado de la variable en cuestión para obtener otro tipo de conclusiones.


#### Análisis Bivariado

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.ion() 

# Supongamos que tienes un DataFrame llamado 'df' con las variables categóricas 'variable1' y 'variable2'

# Crear el gráfico de barras bivariado
plt.figure(figsize=(10, 6))
sns.countplot(data=train_data, x='Type', hue='AdoptionSpeed')
plt.xlabel('Type')
plt.ylabel('Frecuencia')
plt.title('Gráfico de Barras Bivariado')
plt.legend(title='AdoptionSpeed', loc='upper right')  # Agregar leyenda

# Mostrar el gráfico
plt.tight_layout()
plt.show()


C:\Users\Elemir Celis Mendoza\AppData\Local\Temp\ipykernel_18680\4182655612.py:18: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Etapa 2: Primer Corrida

Se generan los primeros modelos () sin agregado de informacion. Dividimos los datos en train y test que luego mantendremos durante todo el notebook. A estos conjuntos se agregara informacion con la intencion de mejorar la prediccion. Entonces, evaluaremos resultados en train, test y con modelos anteriores que tendran menos variables.

Se compararan cuatros modelos:

1. Ridge Classifier
2. K Neighbors Classifier
3. Extra Trees Classifier
4. Random Forest Classifier

### Librerias necearias para esta etapa

In [9]:
# 1. Importo librerias
import os
import numpy as np 
import pandas as pd 

#Librerias para graficos
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme() 

#import cv2 #Para cargar/mostrar/procesar imagenes
import json
import glob #Para listar los archivos en una carpeta

# Librerias propias (contienen los modelos)
import model_metadata
import model_sentiment

# Librerias necesarias para modelos de clasificacion
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

### Funciones creadas

In [10]:
def get_X_y_train(train_data, train_idx):
    X = train_data.drop("AdoptionSpeed", axis=1).iloc[train_idx]
    y = train_data["AdoptionSpeed"].iloc[train_idx]
    return X, y

def get_X_y_val(train_data, val_idx):
    X = train_data.drop("AdoptionSpeed", axis=1).iloc[val_idx]
    y = train_data["AdoptionSpeed"].iloc[val_idx]
    return X, y

#Variable global


# Crear un diccionario para mantener los codificadores por columna
CODIFICADORES = {}



def get_data_prep(train_data):

    #Borrado de columnas
    data_prep = train_data.drop("Description", axis=1)
    #train_data = train_data.drop("Name", axis=1)

    #Encoder para variables discretas
    columnas_encoder = ['Name', 'RescuerID']
    for columna in columnas_encoder:
        encoder = LabelEncoder()
        data_prep[columna] = encoder.fit_transform(data_prep[columna])
        CODIFICADORES[columna] = encoder

    return data_prep

def evaluate_model(y_true, y_pred, set_name, model_name):
    rta = {}

    rta['set_name'] = set_name
    rta['model_name'] = model_name
    rta['accuracy_average_weighted'] = accuracy_score(y_true, y_pred)
    rta['precision_average_weighted'] = precision_score(y_true, y_pred, average='weighted')
    rta['recall_average_weighted'] = recall_score(y_true, y_pred, average='weighted')
    rta['f1_average_weighted'] = f1_score(y_true, y_pred, average='weighted')
    rta['accuracy'] = accuracy_score(y_true, y_pred)
    rta['precision'] = precision_score(y_true, y_pred, average=None)
    rta['recall'] = recall_score(y_true, y_pred, average=None)
    rta['f1'] = f1_score(y_true, y_pred, average=None)

    return rta

def get_feature_importances(data, modelos):
    '''en desarrollo'''
    #RidgeClassifier
    rta=[]
    for modelo in modelos:
        if type(modelo).__name__ == 'RidgeClassifier':
            feature_importances = modelo.coef_[0]
        elif type(modelo).__name__ == 'RandomForestClassifier':
            feature_importances = modelo.feature_importances_
        elif type(modelo).__name__ == 'ExtraTreesClassifier':
            feature_importances = modelo.feature_importances_
        else:
            continue
        
        rta_model = []
        for feature, importance in zip(data.columns, feature_importances):
            rta_model.append((feature, importance))

        rta.append(sorted(rta_model, key=lambda x: abs(x[1]), reverse=True))

    return rta

#Esta funcion importa los archivos json a una lista de objetos. Para luego generar las variables.
def get_object_model(model, listFileFullPath):
    '''
        model = clase del modelo que se quiere cargar (objeto)
        listFullPath = ruta completa de los archivos a cargar (List[string])
    '''

    listObject = []
    for file in listFileFullPath:
        
        file_json = None
        objectModel = None
        
        try:
            with open(file, 'r', encoding='utf-8') as archivo:  # Especifica la codificación utf-8 o la que corresponda
               file_json = json.load(archivo)
        except UnicodeDecodeError as e:
            print(f"Error de decodificación en el archivo '{file}': {e}")
            continue  # Continúa con el siguiente archivo en caso de error
        
        
        fileName = file.split('/')[-1]
        file_json['fileName'] = fileName
        file_json['PetID'] = fileName.split('.')[0].split('-')[0]
        objectModel = model.Model(**file_json)
        listObject.append(objectModel)

    return listObject 

def get_fit_transform(X_train, y_train, X_val, y_val, modelos):
    resultados = []
    for modelo in modelos:
    
        modelo.fit(X_train, y_train)
        y_train_pred = modelo.predict(X_train)
        y_val_pred = modelo.predict(X_val)
        #rta = evaluate_model(y_train, y_train_pred, "Training", type(modelo).__name__)
        #rta.append(get_feature_importances(modelo))
        resultados.append(evaluate_model(y_train, y_train_pred, "Training", type(modelo).__name__))
        resultados.append(evaluate_model(y_val, y_val_pred, "Validation", type(modelo).__name__))

    return resultados
HIPER = []
def get_fit_transform_CV(X_train, y_train, X_val, y_val, modelos, params_grids):

    resultados = []

    for modelo,param_grid  in zip(modelos, params_grids):
        print(type(modelo).__name__)
        grid = GridSearchCV(estimator = modelo, param_grid = param_grid,
                            cv = 5, n_jobs = -1, verbose = 0)
        grid.fit(X_train, y_train)
        HIPER.append((grid.best_estimator_, type(modelo).__name__))
        y_train_pred = grid.best_estimator_.predict(X_train)
        y_val_pred = grid.best_estimator_.predict(X_val)
        resultados.append(evaluate_model(y_train, y_train_pred, "Training", type(modelo).__name__))
        resultados.append(evaluate_model(y_val, y_val_pred, "Validation", type(modelo).__name__))

    return resultados

### Division Train y Valid

In [11]:
#Lectura Dataset inicial 
TRAIN_DATA = pd.read_csv('../data/train/train.csv', index_col='PetID') 
# Divide tus datos en conjuntos de entrenamiento y validación. Mantenemos siempre los mismos indices
train_idx, val_idx = train_test_split(range(len(TRAIN_DATA)), test_size=0.2, random_state=42)


print(f"Train: {len(train_idx)}")
print(f"Val: {len(val_idx)}")

Train: 11994
Val: 2999


### Primera corrida

In [12]:
# Inicializa los modelos
ridge_classifier = RidgeClassifier()
kneighbors_classifier = KNeighborsClassifier()
extra_trees_classifier = ExtraTreesClassifier()
random_forest_classifier = RandomForestClassifier()
#Modelos a evaluar
MODELOS = [ridge_classifier, kneighbors_classifier, extra_trees_classifier, random_forest_classifier]
#Resultados de las distintas iteraciones que se vayan realizando
RESULTADOS = []
PARAMS = []
FEATURE_IMPORTANCES = []

#Primera iteracion con los datos como vienen.
data = get_data_prep(TRAIN_DATA)
X_train, y_train = get_X_y_train(data, train_idx)
X_val, y_val = get_X_y_val(data, val_idx)

print(f"Train: {X_train.shape}")
print(f"Val: {y_train.shape}")
print(f"Train: {X_val.shape}")
print(f"Val: {y_val.shape}")

RESULTADOS.append(get_fit_transform(X_train, y_train, X_val, y_val, MODELOS))
FEATURE_IMPORTANCES.append(get_feature_importances(data, MODELOS))
PARAMS.append(PARAMS.append([modelo.get_params() for modelo in MODELOS]))

del data

Train: (11994, 21)
Val: (11994,)
Train: (2999, 21)
Val: (2999,)


c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

## Optimizacion de hiperparametros

In [13]:
PARAM_GRID = [
{
    'alpha': [0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    #'normalize': [True, False]
},
{
    'leaf_size': [20, 25, 30, 35, 40],
    'metric': ['minkowski', 'euclidean'],
    'distance': ['uniform', 'distance'],
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
},
{
    'n_estimators': [25, 50, 80, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
},
{
    'n_estimators': [25, 50, 80, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

]

In [14]:
RESULTADOS.append(get_fit_transform_CV(X_train, y_train, X_val, y_val, MODELOS, PARAM_GRID))

RidgeClassifier


c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Elemir Celis Mendoza\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

KNeighborsClassifier


# 4 - Buscar el modelo optimo

Se realizara un proceso evolutivo, donde iremos agregando variables y re-evaluando los modelos. Con la intencion de verificar si existe mejora o no con el agregar de las variables. Para esto se evaluaran las mismas metricas vistas anteriormente.



## 4.1 - Sentimientos de las descripciones

Informacion disponible de los comentarios:

1. magnitude.
2. score.
3. Cantidad de sentencias.
4. EntidadCantidad.
5. Entidad por tipo (son varias columnas).
6. Idioma.

In [ ]:
# Generacion de funciones para obtener las nuevas variables

getPetID = lambda x: x.PetID
getPetID_vec = np.vectorize(getPetID)

#Solo para la modelo model_sentiment
#1. magnitude
getMagnitude = lambda x: x.documentSentiment.magnitude
getMagnitude_vec = np.vectorize(getMagnitude)

#2. score
getScore = lambda x: x.documentSentiment.score
getScore_vec = np.vectorize(getScore)

#3. Cantidad de sentencias
getLenEntity = lambda x: len(x.entities)
getLenEntity_vec = np.vectorize(getLenEntity)

#4. EntidadCantidad
getLenSentences = lambda x: len(x.sentences)
getLenSentences_vec = np.vectorize(getLenSentences)

# 5. Entidad por tipo (son varias columnas)
getEntitiesCount = lambda x: x.get_entity_count()
getEntitiesCount_vec = np.vectorize(getEntitiesCount)

# 6. Idioma (despues hay que hacer un encoder)
# Es necesario hacerle un encoder a esta variable
getLenguage = lambda x: x.language
getLenguage_vec = np.vectorize(getLenguage)



## Lectura de los archivos sentiment

In [ ]:
#Lectura de archivos sentiments
f_train_sentiments = glob.glob('../data/train_sentiment/*.json')
f_train_sentiments= [ruta.replace('\\', '/') for ruta in f_train_sentiments]
LIST_SENTIMENTS = get_object_model(model_sentiment, f_train_sentiments)
del f_train_sentiments

#Se agrega informacion de los archivo sentiment
#Si tiene o no archivo sentiment
data = get_data_prep(TRAIN_DATA)
data['sentiment?'] = data.index.isin(getPetID_vec(LIST_SENTIMENTS))


df = pd.DataFrame(
        data = {'Score' : getScore_vec(LIST_SENTIMENTS).tolist()
                ,'Magnitude' : getMagnitude_vec(LIST_SENTIMENTS).tolist()
                ,'CountEntity' : getLenEntity_vec(LIST_SENTIMENTS).tolist()
                ,'CountSentences' : getLenSentences_vec(LIST_SENTIMENTS).tolist()
                ,'EntitiesCount' : getEntitiesCount_vec(LIST_SENTIMENTS).tolist()
                ,'Lenguage': getLenguage_vec(LIST_SENTIMENTS).tolist()
                }
        , index=getPetID_vec(LIST_SENTIMENTS))

df_EntitiesCount = (df['EntitiesCount'].apply(lambda x: pd.Series(x, dtype=float))).fillna(0).astype(int)
df_EntitiesCount.columns = ['EntitiesCount_' + str(col) for col in df_EntitiesCount.columns]

df.drop('EntitiesCount', axis=1, inplace=True)
df.index.name = 'PetID'
#agrego las columnas de EntitiesCount
df = df.join(df_EntitiesCount, on='PetID', how='left', lsuffix='_left', rsuffix='_right')



df = data.join(df, on='PetID', how='left', lsuffix='_left', rsuffix='_right')

del df_EntitiesCount
del df
del LIST_SENTIMENTS


#Segunda iteracion agregando las variables de sentiment
X_train, y_train = get_X_y_train(data, train_idx)
X_val, y_val = get_X_y_val(data, val_idx)

RESULTADOS.append(get_fit_transform(X_train, y_train, X_val, y_val, MODELOS))

del data